<a href="https://colab.research.google.com/github/kiryu-3/Prmn2023/blob/main/Python/Python_Machine/Machine_Learning_7_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 多値分類ー知識編

In [ ]:
# 最初にインポートしてください
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge, Lasso, LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, log_loss
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
import statsmodels.api as sma

## 多値分類

目的変数を3つ以上の値として扱う分類アルゴリズムを、**多値分類**といいます。

通常のロジスティック回帰は二値分類にしか対応していないため、   
多クラス分類に応用するには工夫が必要です。



## OnevsRest

多値分類を行うための一つの方法が、**One vs Rest(OvR)**です。

一つのクラスとそれ以外の全クラスの二値分類器を、  
クラスの数だけ作って最終的に多値分類にするやり方になります。

下の例の場合、三つの分類器の結果のうち最も高い確率のクラスを予測結果とします。  


（参考：https://shorturl.at/iqF01 ）

![](https://imgur.com/idrfO9b.png)


- ロジスティック回帰に限らず、他のモデルでも使われる最も一般的な手法である
- データ数が多かったり、
モデル構築にコストがかかる場合は、あまり好まれない

## 多項ロジスティック回帰

多値分類を行うためのもう一つの方法が、**多項ロジスティック回帰(Multinomial logistic regression)**です。

ロジスティック回帰の損失関数を、多クラスにも適用できるように変更したやり方になります。  
一度の学習で多クラスに対応するモデルを構築可能です。


以下のように応用していくことになります。

- 目的変数のエンコーディング
- シグモイド関数 → ソフトマックス関数
- 損失関数の多クラス対応

### 目的変数のエンコーディング

カテゴリーのとりうる値の数だけを0と1で構成するベクトルを  
**one-hot ベクトル(one-hot encoded vector)**といいます。

このベクトルを使って、そのカテゴリーだけを1(hot)で表す処理を行います（**one-hot エンコード**）。


多値分類では、二値分類のように目的変数を0,1,2...のようなラベル付けをしません。

単純に数値化できないので、目的変数のone-hot エンコーディングが必要になるわけです。

以下の図のようなイメージです。

![](https://imgur.com/yiMEC8E.png)

説明変数ではなく目的変数のエンコーディングなので、ダミー変数トラップを意識する必要はありません。

数式で表す場合、以下のように定義できます。
- $i$：$i$番目のデータ
- $k$：$k$番目のクラス

$$
\begin{equation}
t_{ik}=\left\{
\begin{aligned}
1 \qquad\text{if}\:y_i=k\\
0 \qquad\text{if}\:y_i\neq k\\
\end{aligned}
\right.
\end{equation}
$$

### ソフトマックス関数

二値分類ではシグモイド関数を使っていましたが、  
多値分類では**ソフトマックス関数(softmax function)**を使います。

数式は以下の形になります。

$$
p_k(x_i)=\frac{e^{x_i}}{e^{x_1}+e^{x_2}+\cdots+e^{x_K}}=\frac{e^{x_i}}{\sum_{k=1}^{K}e^{x_k}}
$$

$K$はクラス数です。  
この値は、各クラスの出現確率として扱うことができます。

$K=2$, $x_2=0$とすると、シグモイド関数の形と同じになります。

このことから、ソフトマックス関数はシグモイド関数を一般化したものであるといえます。


特徴量が一つの場合、$x_k$は線形回帰モデルの出力結果なので、$\theta^k_0+\theta_1^kx$の形をとります。

ソフトマックス関数に代入すると以下のようになります。

$$
p_k(x_i)=\frac{e^{\theta^i_0+\theta_1^ix}}{\sum_{k=1}^{K}e^{\theta^k_0+\theta_1^kx}}
$$



ソフトマックス関数を使えば、どんな値でも最終的に確率の形にすることができます。

![](https://imgur.com/yPL6efV.png)

単純にそれぞれの割合を計算しているわけではないことに注意してください。  
指数関数をとっていることが重要です。

### 損失関数

多項ロジスティック回帰における損失関数は、  
**交差エントロピー**を多クラス対応できるように一般化したものになります。



データ$i$における損失を$Cost(p(x_i), y_i)$とすると、  
交差エントロピーは以下のように表されます。

$$
\begin{equation}
Cost(p(x_i), y_i)=\left\{
\begin{aligned}
-log(p(x_i)) & \qquad\text{if}\:y_i=1\\
-log(1-p(x_i)) & \qquad\text{if}\:y_i=0\\
\end{aligned}
\right.
\end{equation}
$$


これは二値分類を前提にしています。  
代わりに$t_{ik}$を使います。

$$
\begin{equation}
t_{ik}=\left\{
\begin{aligned}
1 \qquad\text{if}\:y_i=k\\
0 \qquad\text{if}\:y_i\neq k\\
\end{aligned}
\right.
\end{equation}
$$

これを使って、多値分類の損失を表現すると以下のようになります。

$$
Cost(p(x_i), y_i)=-t_{ik}log(p_k(x_i))
$$

二値分類のときと同様、正解ラベル以外の出力結果は損失としてカウントしません。

![](https://imgur.com/7A1pStl.png)

この損失をすべてのデータで平均（あるいは合計）を  
取ったものが損失関数になります。

$$
L(\theta)=\frac{1}{m}\sum^{m}_{i=1}Cost(p(x_i), y_i)=-\frac{1}{m}\sum^{m}_{i=1}\sum^{K}_{k=1}t_{ik}log(p_k(x_i))
$$

- $\theta=\theta_0, \theta_1\cdots\theta_n$
- $i$：〇番目のデータ
- $m$：データの総数
- $K$：クラス数


### パラメータの最適化

**最急降下法**は、最も急になる方向に少しずつパラメータを動かして最適解を探る方法です。

機械学習ではこのように、パラメータを徐々に最適解に近づけていくことを  
**「学習する」**といいます。

線形回帰のときと同様、$L(\theta)$のパラメータを以下の式で同時に更新していきます。

$$
\theta_0 := \theta_0 – \alpha\frac{\partial}{\partial\theta_0}L(\theta)
$$

$$
\theta_1 := \theta_1 – \alpha\frac{\partial}{\partial\theta_1}L(\theta)
$$

$$
・
$$
$$  
・  
$$
$$  
・  
$$

$$
\theta_n := \theta_n – \alpha\frac{\partial}{\partial\theta_n}L(\theta)
$$

微分において、一旦シグマは置いて考えます。

$$
E = -t_{ik}log(p_k(x_i))
$$

また、$j$番目の入力$\theta_0+\theta_{1}x_{1}+\dots+\theta_{n}x_{n}$を$a_j$と置き換えます。

以下の計算ができれば、パラメータの更新が行えそうです。

$$
\frac{\partial}{\partial\theta_n}L(\theta_n) = \frac{\partial a_j}{\partial \theta_n}\ \frac{\partial p_k(x_i)}{\partial a_j}\ \frac{\partial E}{\partial p_k(x_i)}
$$

#### 交差エントロピー（多項式）の微分



$$
{\begin{align}
\frac{\partial E}{\partial p_k(x_i)}\
&= \frac{\partial }{\partial p_k(x_i)}\ \Bigl( -t_{ik}log(p_k(x_i)) \Bigr) \\
&= -\frac{t_{ik}}{p_k(x_i)}
\end{align}
}
$$

#### ソフトマックス関数の微分

$$
{\begin{align}
\frac{\partial p_k(x_i)}{\partial a_j}\
&= \frac{\partial }{\partial a_j}\ \Bigl(\frac{e^{a_i}}{e^{a_1}+e^{a_2}+\cdots+e^{a_K}} \Bigr) \\
&= \frac{\partial }{\partial a_j}\ \Bigl(\frac{e^{a_i}}{Z}
\Bigr) \\
\end{align}
}
$$




$i=j$のとき、

$$
{\begin{align}
\frac{\partial p_k(x_i)}{\partial a_j}\
&= \frac{\partial }{\partial a_i}\ \Bigl(\frac{e^{a_i}}{Z} \Bigr) \\
&= \frac{e^{a_i}Z - e^{a_i}e^{a_i}}{Z^2}\\
&= p_k(x_i)(1-p_k(x_i))\\
\end{align}
}
$$

$i≠j$のとき、

$$
{\begin{align}
\frac{\partial p_k(x_i)}{\partial a_j}\
&= \frac{\partial }{\partial a_j}\ \Bigl(\frac{e^{a_i}}{Z} \Bigr) \\
&= \frac{- e^{a_i}e^{a_j}}{Z^2}\\
&= -p_k(x_i)p_k(x_j)\\
\end{align}
}
$$

これらをまとめると、

$$
\begin{equation}
\frac{\partial p_k(x_i)}{\partial a_j}=\left\{
\begin{aligned}
p_k(x_i)(1-p_k(x_i)) & \qquad(i=j)\\
-p_k(x_i)p_k(x_j) & \qquad(i≠j)\\
\end{aligned}
\right.
\end{equation}
$$

実際は、$\delta$を使って以下のようにまとめることが多いです。

$$
\frac{\partial p_k(x_i)}{\partial a_j}=p_k(x_i)(\delta_{ij}-p_k(x_j))
$$

ただし、
$
\begin{equation}
\delta_{ij}=\left\{
\begin{aligned}
1 \qquad\text{if}\:i=j\\
0 \qquad\text{if}\:i\neq j\\
\end{aligned}
\right.
\end{equation}
$

#### 線形結合の微分

$$
\begin{equation}
\frac{\partial a_j}{\partial \theta_n}=\left\{
\begin{aligned}
x_n & \qquad\text{if}\:n≠0\\
1 & \qquad\text{if}\:n=0\\
\end{aligned}
\right.
\end{equation}
$$

#### 偏微分結果

$$
{\begin{align}
\frac{\partial}{\partial\theta_n}L(\theta_n)
&= \frac{\partial a_j}{\partial \theta_n}\ \frac{\partial p_k(x_i)}{\partial a_j}\ \frac{\partial E}{\partial p_k(x_i)} \\
&= \frac{\partial a_j}{\partial \theta_n}\ p_k(x_i)(\delta_{ij}-p_k(x_i)) \Bigl(-\frac{t_{ik}}{p_k(x_i)}\Bigr) \\
&= \frac{\partial a_j}{\partial \theta_n}t_{ik}(p_k(x_i)-\delta_{ij}) \\
&= \frac{\partial a_j}{\partial \theta_n}t_{ik}(p_k(x_i)-1) + \frac{\partial a_j}{\partial \theta_n}t_{ik}p_k(x_i) \\
\end{align}
}
$$

ここからさらに変形します。

$$
{\begin{align}
\frac{1}{m}\sum^{m}_{i=1}\sum^{K}_{k=1}\frac{\partial a_j}{\partial \theta_n}t_{ik}(p_k(x_i)-\delta_{ij})
&= \frac{1}{m}\sum^{m}_{i=1}\sum^{K}_{k=1}\frac{\partial a_j}{\partial \theta_n}t_{ik}p_k(x_i)- \frac{1}{m}\sum^{m}_{i=1}\sum^{K}_{k=1}\frac{\partial a_j}{\partial \theta_n}t_{ik}\delta_{ij} \\
\end{align}
}
$$

$\sum^{m}_{i=1}\sum^{K}_{k=1}t_{ik}$ は、どこかでは必ず$y_i=k$となるため、  
その値は1になるはずです。

ただし、$\sum^{m}_{i=1}\sum^{K}_{k=1}\delta_{ij}$は値自体は1となりますが、  
$i=j$の場合に1となるだけなので、$t_{ik}$はこれ以上簡単化できません。  
（$y_i=k$と$i=j$という二つの条件を考えなければならない）

よって、偏微分の結果は以下のようになります。  
実際の$\theta$は$\theta=\theta^1_0, \cdots, \theta^K_0,　\cdots,　 \theta^1_n, \cdots, \theta^K_n$と複数あることに注意が必要です。  
（$n≠0$）

$$
\frac{\partial L(\theta)}{\partial\theta^k_0}=\frac{1}{m}\sum^{m}_{i=1}(p_k(x_i)-t_{ik})
$$


$$
\frac{\partial L(\theta)}{\partial\theta^k_n}=\frac{1}{m}\sum^{m}_{i=1}(p_k(x_i)-t_{ik})x_i
$$

#### 最終式

$$
\theta_0 := \theta_0 – \alpha\frac{1}{m}\sum^{m}_{i=1}\Bigl(\frac{e^{x_i}}{\sum_{k=1}^{K}e^{x_k}}-t_{ik}\Bigr)
$$

$$
\theta_1 := \theta_1 – \alpha\frac{1}{m}\sum^{m}_{i=1}\Bigl(\frac{e^{x_i}}{\sum_{k=1}^{K}e^{x_k}}-t_{ik}\Bigr)x_i
$$

$$
・
$$
$$  
・  
$$
$$  
・  
$$

$$
\theta_n := \theta_n – \alpha\frac{1}{m}\sum^{m}_{i=1}\Bigl(\frac{e^{x_i}}{\sum_{k=1}^{K}e^{x_k}}-t_{ik}\Bigr)x_i
$$

### Pythonで多値分類

では、実際にPythonで多値分類を行っていきましょう。


#### 使用するデータセット

今回は、seabornのサンプル用データセット"tips"を利用します。

データセットは、pandas.DataFrameオブジェクトとして取得することができます。

（参考サイト：[こちら](https://biotech-lab.org/articles/1408#i)）

In [ ]:
tips = sns.load_dataset('tips')
tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


"tips"のデータの説明は以下の通りです。

- total_bill : 総支払額(食事代、税込み)　(USドル)
- tip : チップ(USドル)
- sex : 性別
- smoker : 喫煙者か否か
- day : 曜日(木・金・土・日のいずれか)
- time : 食事の時間(昼食か夕食か)
- size : 人数

また、今回のデータには欠損値はなさそうです。

#### Pythonによる実行(OvR)

今回は使用するカラムを増やしたいので、one-hotエンコーディングを行います。

In [ ]:
# day カラム以外のカテゴリカル変数をダミー変数に変換
tips = pd.get_dummies(tips, columns=["sex", "smoker", "time"], drop_first=True)
tips.head()

,total_bill,tip,day,size,sex_Female,smoker_No,time_Dinner
0,16.99,1.01,Sun,2,1,1,1
1,10.34,1.66,Sun,3,0,1,1
2,21.01,3.50,Sun,3,0,1,1
3,23.68,3.31,Sun,2,0,1,1
4,24.59,3.61,Sun,4,1,1,1


今回は、"day"カラムの値（何曜日かどうか）を予測するとします。  

まずは、曜日ごとの人数を割り出しましょう。


In [ ]:
tips["day"].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

まずは、OvRを使って学習を行っていきましょう。

本当はk-fold法（参考：[こちら](https://datawokagaku.com/kfoldcv/)）を使って汎化性能を測った方がいいのですが、  
ここでは説明のしやすさのためhold-out法を使います。

In [ ]:
# 目的変数に"day"、説明変数にそれ以外のカラムを指定
x = tips.drop('day', axis=1)
y = tips['day']

# データを訓練データとテストデータに分ける
# データのばらつき具合を実データに合わせる
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0, stratify=y)

# 説明変数のデータを標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model_ovr = LogisticRegression(penalty='none', multi_class='ovr')
model_ovr.fit(x_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


LogisticRegression(multi_class='ovr', penalty='none')

それぞれのラベルの分類結果を取得したいときは、  
**`.predict(X)`**を使います。

In [ ]:
y_pred = model_ovr.predict(x_test_scaled)
y_pred[:9]

array(['Sat', 'Sat', 'Sat', 'Sat', 'Sun', 'Sun', 'Sun', 'Sun', 'Thur'],
      dtype=object)

それぞれのラベルの確率$p(X)$を取得したいときは、  
**`.predict_proba(X)`**を使います。

In [ ]:
y_pred_ovr = model_ovr.predict_proba(x_test_scaled)
print(model_ovr.classes_)  # クラスの値
print(model_ovr.intercept_)  # 切片の値
print(y_pred_ovr[:9])  # それぞれの出現確率

['Fri' 'Sat' 'Sun' 'Thur']
[-3.11653006 -5.40578074 -5.61625749 -5.19689633]
[[1.97321539e-01 6.13879239e-01 1.88799222e-01 2.68153201e-11]
 [2.16487947e-01 5.88394769e-01 1.95117284e-01 9.14947740e-12]
 [1.61248088e-01 6.72221388e-01 1.66530523e-01 2.77217780e-10]
 [1.69772464e-01 5.96155809e-01 2.34071726e-01 4.49290769e-11]
 [1.79213405e-02 4.02513973e-01 5.74134335e-01 5.43035151e-03]
 [4.21007563e-02 4.67954147e-01 4.83853377e-01 6.09172035e-03]
 [4.15021584e-03 3.40036140e-01 6.03192149e-01 5.26214949e-02]
 [1.20132272e-02 3.60810960e-01 6.03684252e-01 2.34915607e-02]
 [4.33358370e-02 1.97928620e-09 2.50188309e-09 9.56664159e-01]]


係数の値を確認してみましょう。

In [ ]:
# coefficient  係数
print(pd.Series(model_ovr.coef_[0], index=x.columns))

total_bill    -0.172670
tip           -0.080569
size          -0.795666
sex_Female    -0.072304
smoker_No     -0.878311
time_Dinner   -0.034608
dtype: float64


#### 交差エントロピーの導出(OvR)

学習(fit)時には、学習データにおける交差エントロピーを最小にするように、  
パラメータ$\theta$を求めています。

その後、テストデータを使って交差エントロピーを求めたい場合、  
`log_loss(y_test, y_pred_proba)`を使って求めることができます。

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, y_pred_ovr)

0.7017074026653042

この値の値が小さければ小さいほど、優秀なモデルであると言えます。    
今回はこの数字の解釈はしません。

パラメータを変更することで精度向上が期待できます。  
パラメータについては[こちら](https://datawokagaku.com/logstic_reg_python/#i-3)が参考にしてください。

#### Pythonによる実行(多項ロジスティック回帰)

次に、多項ロジスティック回帰を使って学習を行っていきましょう。

本当はk-fold法（参考：[こちら](https://datawokagaku.com/kfoldcv/)）を使って汎化性能を測った方がいいのですが、  
ここでは説明のしやすさのためhold-out法を使います。

In [ ]:
# 目的変数に"day"、説明変数にそれ以外のカラムを指定
x = tips.drop('day', axis=1)
y = tips['day']

# データを訓練データとテストデータに分ける
# データのばらつき具合を実データに合わせる
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0, stratify=y)

# 説明変数のデータを標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model_mn = LogisticRegression(penalty='none', multi_class='multinomial')
model_mn.fit(x_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


LogisticRegression(multi_class='multinomial', penalty='none')

それぞれのラベルの分類結果を取得したいときは、  
**`.predict(X)`**を使います。

In [ ]:
y_test[:9]

190     Sun
218     Sat
237     Sat
106     Sat
38      Sat
242     Sat
52      Sun
39      Sat
145    Thur
Name: day, dtype: category
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [ ]:
y_pred = model_mn.predict(x_test_scaled)
y_pred[:9]

array(['Sat', 'Sat', 'Sat', 'Sat', 'Sun', 'Sun', 'Sun', 'Sun', 'Thur'],
      dtype=object)

それぞれのラベルの確率$p(X)$を取得したいときは、  
**`.predict_proba(X)`**を使います。

In [ ]:
y_pred_mn = model_mn.predict_proba(x_test_scaled)
print(model_mn.classes_)  # クラスの値
print(model_mn.intercept_)  # 切片の値
print(y_pred_mn[:9])  # それぞれの出現確率

['Fri' 'Sat' 'Sun' 'Thur']
[ 1.87222867 -0.79407369 -0.4497952  -0.62835978]
[[1.70259668e-01 6.35448936e-01 1.94291396e-01 9.43492608e-11]
 [1.63342008e-01 6.32572766e-01 2.04085226e-01 2.46751776e-11]
 [1.93972950e-01 6.40028519e-01 1.65998528e-01 1.79183839e-09]
 [1.34569527e-01 6.23996807e-01 2.41433666e-01 1.53507948e-10]
 [2.37351691e-02 3.98835236e-01 5.72139449e-01 5.29014544e-03]
 [6.90835769e-02 4.56018199e-01 4.64069242e-01 1.08289823e-02]
 [6.85743652e-03 3.35560792e-01 6.18708327e-01 3.88734443e-02]
 [1.72153964e-02 3.48996008e-01 6.05658316e-01 2.81302793e-02]
 [4.26949711e-09 2.80391236e-16 1.89348135e-15 9.99999996e-01]]


係数の値を確認してみましょう。

In [ ]:
# coefficient  係数
print(pd.Series(model_mn.coef_[0], index=x.columns))

total_bill    -0.335634
tip           -0.095471
size          -0.565336
sex_Female    -0.025154
smoker_No     -2.603051
time_Dinner   -1.399636
dtype: float64


#### 交差エントロピーの導出(多項ロジスティック回帰)

学習(fit)時には、学習データにおける交差エントロピーを最小にするように、  
パラメータ$\theta$を求めています。

その後、テストデータを使って交差エントロピーを求めたい場合、  
`log_loss(y_test, y_pred_proba)`を使って求めることができます。

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, y_pred_mn)

0.9231915699705989

この値の値が小さければ小さいほど、優秀なモデルであると言えます。    
今回はこの数字の解釈はしません。

パラメータを変更することで精度向上が期待できます。  
パラメータについては[こちら](https://datawokagaku.com/logstic_reg_python/#i-3)が参考にしてください。

#### OvRと多項ロジスティック回帰の比較

それぞれの出現回数を比較してみましょう。

In [ ]:
print(model_ovr.classes_)  # クラスの値
print(model_ovr.intercept_)  # 切片の値
print(y_pred_ovr[:9])  # それぞれの出現確率

['Fri' 'Sat' 'Sun' 'Thur']
[-3.11653006 -5.40578074 -5.61625749 -5.19689633]
[[1.97321539e-01 6.13879239e-01 1.88799222e-01 2.68153201e-11]
 [2.16487947e-01 5.88394769e-01 1.95117284e-01 9.14947740e-12]
 [1.61248088e-01 6.72221388e-01 1.66530523e-01 2.77217780e-10]
 [1.69772464e-01 5.96155809e-01 2.34071726e-01 4.49290769e-11]
 [1.79213405e-02 4.02513973e-01 5.74134335e-01 5.43035151e-03]
 [4.21007563e-02 4.67954147e-01 4.83853377e-01 6.09172035e-03]
 [4.15021584e-03 3.40036140e-01 6.03192149e-01 5.26214949e-02]
 [1.20132272e-02 3.60810960e-01 6.03684252e-01 2.34915607e-02]
 [4.33358370e-02 1.97928620e-09 2.50188309e-09 9.56664159e-01]]


In [ ]:
y_pred_mn = model_mn.predict_proba(x_test_scaled)
print(model_mn.classes_)  # クラスの値
print(model_mn.intercept_)  # 切片の値
print(y_pred_mn[:9])  # それぞれの出現確率

['Fri' 'Sat' 'Sun' 'Thur']
[ 1.87222867 -0.79407369 -0.4497952  -0.62835978]
[[1.70259668e-01 6.35448936e-01 1.94291396e-01 9.43492608e-11]
 [1.63342008e-01 6.32572766e-01 2.04085226e-01 2.46751776e-11]
 [1.93972950e-01 6.40028519e-01 1.65998528e-01 1.79183839e-09]
 [1.34569527e-01 6.23996807e-01 2.41433666e-01 1.53507948e-10]
 [2.37351691e-02 3.98835236e-01 5.72139449e-01 5.29014544e-03]
 [6.90835769e-02 4.56018199e-01 4.64069242e-01 1.08289823e-02]
 [6.85743652e-03 3.35560792e-01 6.18708327e-01 3.88734443e-02]
 [1.72153964e-02 3.48996008e-01 6.05658316e-01 2.81302793e-02]
 [4.26949711e-09 2.80391236e-16 1.89348135e-15 9.99999996e-01]]


今回"Thur"のデータは、"Sat"や"Sun"のデータに比べると少ないです。

In [ ]:
# 予測結果 y_pred の各クラスの出現回数を求める
y_pred = model_ovr.predict(x_test_scaled)
unique_classes, counts = np.unique(y_pred, return_counts=True)

# クラスと出現回数を表示
for cls, count in zip(unique_classes, counts):
    print(f"クラス {cls}: {count}回")

クラス Sat: 25回
クラス Sun: 27回
クラス Thur: 22回


In [ ]:
# 予測結果 y_pred の各クラスの出現回数を求める
y_pred = model_mn.predict(x_test_scaled)
unique_classes, counts = np.unique(y_pred, return_counts=True)

# クラスと出現回数を表示
for cls, count in zip(unique_classes, counts):
    print(f"クラス {cls}: {count}回")

クラス Sat: 25回
クラス Sun: 27回
クラス Thur: 22回


今回はこの"Thr"であると予測したデータについてになりますが、  
明らかにそのクラスだというデータに対しては、出力結果が変わります。

In [ ]:
print(model_mn.classes_)  # クラスの値
print(y_pred_ovr[8])  # 出現確率(OvR)
print(y_pred_mn[8])  # 出現確率(多項ロジスティック回帰)

['Fri' 'Sat' 'Sun' 'Thur']
[4.33358370e-02 1.97928620e-09 2.50188309e-09 9.56664159e-01]
[4.26949711e-09 2.80391236e-16 1.89348135e-15 9.99999996e-01]


OvRのときは、3つのモデルを作って最後確率に戻したりしている関係で、  
それぞれのクラスの確率は比較的ならされています。

一方、多項ロジスティック回帰のときは、一つのモデルで直接学習をしている関係で、
明らかにそのモデルだとわかった時は確率値が極端になりがちです。